# Asymetric semantic search
## Bi-encoder (RuBert) Fine-tuning using Sentence-Transformers library

dataset: https://huggingface.co/datasets/sberquad
training overview: https://www.sbert.net/docs/training/overview.html

In [6]:
import torch

device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

mps


In [7]:
from sentence_transformers import SentenceTransformer, models


def raw_bi_encoder():
    word_embedding_model = models.Transformer('cointegrated/rubert-tiny2', max_seq_length=256)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    bi_encoder = SentenceTransformer(
        modules=[word_embedding_model, pooling_model],
        device=device,
    )

    return bi_encoder

In [8]:
bi_encoder = raw_bi_encoder()

print(bi_encoder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


# Let's make sure it works

In [9]:
from sentence_transformers import evaluation

document_context = [
    'Город Байконур и космодром Байконур вместе образуют комплекс Байконур , арендованный Россией у Казахстана на период до 2050 года. Эксплуатация космодрома стоит около 9 млрд рублей в год (стоимость аренды комплекса Байконур составляет 115 млн долларов — около 7,4 млрд рублей в год; ещё около 1,5 млрд рублей в год Россия тратит на поддержание объектов космодрома), что составляет 4,2 % от общего бюджета Роскосмоса на 2012 год. Кроме того, из федерального бюджета России в бюджет города Байконура ежегодно осуществляется безвозмездное поступление в размере 1,16 млрд рублей (по состоянию на 2012 год). В общей сложности космодром и город обходятся бюджету России в 10,16 млрд рублей в год.',

    'Скорость света в вакууме — абсолютная величина скорости распространения электромагнитных волн в вакууме. Традиционно обозначается латинской буквой c (произносится как [це]). Скорость света в вакууме — фундаментальная постоянная, не зависящая от выбора инерциальной системы отсчёта (ИСО). Она относится к фундаментальным физическим постоянным, которые характеризуют не просто отдельные тела или поля, а свойства пространства-времени в целом. По современным представлениям, скорость света в вакууме — предельная скорость движения частиц и распространения взаимодействий.',

    'Первый троллейбус был создан в Германии инженером Вернером фон Сименсом, вероятно, под влиянием идеи его брата, проживавшего в Англии доктора Вильгельма Сименса, высказанной 18 мая 1881 года на двадцать втором заседании Королевского научного общества. Электросъём осуществлялся восьмиколёсной тележкой (Kontaktwagen), катившейся по двум параллельным контактным проводам. Провода располагались достаточно близко друг от друга, и при сильном ветре нередко перехлёстывались, что приводило к коротким замыканиям. Экспериментальная троллейбусная линия протяжённостью 540 м (591 ярд), открытая компанией Siemens & Halske в предместье Берлина Галензе (Halensee), действовала с 29 апреля по 13 июня 1882.',
]

question = [
    'На какой период был арендован Россией комплекс Байконур',

    'Как называется абсолютная величина скорости распространения электромагнитных волн в вакууме?',

    'Кем был ослеплен князь Василий Тёмный?',
]

label = [
    1.0,
    1.0,
    0.0,
]

evaluator = evaluation.EmbeddingSimilarityEvaluator(question, document_context, label)
bi_encoder.evaluate(evaluator)

0.8660254037844387

# Evaluate on train-val without Fine-Tuning

In [10]:
from hw4.data_preparer import trainval_shuffled_data

trainval = trainval_shuffled_data(from_file=True)

n_samples = 1000
questions, document_contexts, coss = zip(*trainval[:n_samples])

In [11]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(questions, document_contexts, coss)

In [12]:
bi_encoder = raw_bi_encoder()

In [19]:
bi_encoder.evaluate(evaluator)

0.7866959794370026

# Evaluate on test without Fine-Tuning

In [13]:
from hw4.data_preparer import test_shuffled_data

n_test_samples = 1000
test = test_shuffled_data(from_file=True)
test_questions, test_document_contexts, test_coss = zip(*test[:n_test_samples])

In [14]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(test_questions, test_document_contexts, test_coss)

In [15]:
bi_encoder = raw_bi_encoder()

In [12]:
bi_encoder.evaluate(evaluator)

0.8064889530877336

# Fine-Tuning

In [9]:
from hw4.data_preparer import trainval_examples

trainval_examples = trainval_examples()[:n_samples]
print(trainval_examples[0].__dict__)

{'guid': '', 'texts': ['что оказала Заметное, хотя и менее значительное влияние на воззрения Локка', 'Заметное, хотя и менее значительное влияние на воззрения Локка оказала психология Гоббса, у которого заимствован, например, порядок изложения Опыта . Описывая процессы сравнения, Локк следует за Гоббсом; вместе с ним он утверждает, что отношения не принадлежат вещам, а составляют результат сравнения, что отношений бесчисленное множество, что более важные отношения суть тождество и различие, равенство и неравенство, сходство и несходство, смежность по пространству и времени, причина и действие. В трактате о языке, то есть в третьей книге Опыта , Локк развивает мысли Гоббса. В учении о воле Локк находится в сильнейшей зависимости от Гоббса; вместе с последним он учит, что стремление к удовольствию есть единственное проходящее через всю нашу психическую жизнь и что понятие о добре и зле у различных людей совершенно различно. В учении о свободе воли Локк вместе с Гоббсом утверждает, что во

In [10]:
from torch.utils.data import DataLoader

trainval_dataloader_set = DataLoader(trainval_examples, shuffle=True, batch_size=32,
                                     collate_fn=bi_encoder.smart_batching_collate)

In [11]:
from sentence_transformers import losses

trainval_loss_set = losses.CosineSimilarityLoss(bi_encoder)

In [12]:
trainval_loss_set

CosineSimilarityLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (loss_fct): MSELoss()
  (cos_score_transformation): Identity()
)

In [13]:
(query_batch, context_batch), labels = next(iter(trainval_dataloader_set))
query_batch['input_ids'].shape, context_batch['input_ids'].shape, labels.shape

(torch.Size([32, 33]), torch.Size([32, 256]), torch.Size([32]))

In [17]:
bi_encoder.fit(
    train_objectives=[(trainval_dataloader_set, trainval_loss_set)],
    output_path='qa/results',
    epochs=5,
    evaluator=evaluator,
)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

# Loading model from checkpoint

In [16]:
from sentence_transformers import SentenceTransformer

finetuned_bi_encoder = SentenceTransformer('qa/results')

In [17]:
finetuned_bi_encoder.evaluate(evaluator)

0.8099115488020344

# Evaluating on metrics

In [18]:
from hw4.data_preparer import test_shuffled_data

data = test_shuffled_data(from_file=True)

In [19]:
n_test_samples = 1000
queries, docs, labels = zip(*data[:n_test_samples])

In [20]:
print('docs count before shrinking: ', len(docs))

docs count before shrinking:  1000


In [21]:
docs = list(set(docs))

In [22]:
print('docs count without duplicates: ', len(docs))

docs count without duplicates:  891


In [23]:
doc_embs = finetuned_bi_encoder.encode(
    docs,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

In [24]:
query_embs = finetuned_bi_encoder.encode(
    queries,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [25]:
from sentence_transformers import util

In [26]:
cos_sim_score = util.semantic_search(query_embs, doc_embs, top_k=3)

In [27]:
for_first_query = cos_sim_score[0]
print(for_first_query)

[{'corpus_id': 592, 'score': 0.5318310856819153}, {'corpus_id': 770, 'score': 0.5177189111709595}, {'corpus_id': 420, 'score': 0.5000532269477844}]


In [28]:
print('Question: ', queries[0])
print()
for no, ir in enumerate(cos_sim_score[0]):
    corpus_id = ir["corpus_id"]
    print('corpus id: ', corpus_id)
    print(f'Document {no + 1}: Cosine Similarity is {ir["score"]:.3f}:\n\n{docs[corpus_id]}')
    print()

Question:  Какое содержание селена в морской воде?

corpus id:  592
Document 1: Cosine Similarity is 0.532:

Содержание селена в земной коре — около 500 мг/т. Основные черты геохимии селена в земной коре определяются близостью его ионного радиуса к ионному радиусу серы. Селен образует 37 минералов, среди которых в первую очередь должны быть отмечены ашавалит FeSe, клаусталит PbSe, тиманнит HgSe, гуанахуатит Bi2(Se, S)3, хастит CoSe2, платинит PbBi2(S, Se)3, ассоциирующие с различными сульфидами, а иногда также с касситеритом. Изредка встречается самородный селен. Главное промышленное значение на селен имеют сульфидные месторождения. Содержание селена в сульфидах колеблется от 7 до 110 г/т. Концентрация селена в морской воде 4·10−4 мг/л[15]. На территории Кавказских Минеральных Вод встречаются источники с содержанием Se от 50 мкг/дм3.

corpus id:  770
Document 2: Cosine Similarity is 0.518:

К хлоридно-гидрокарбонатно-натриевому типу относится закарпатская вода Драговская с минерализаци

In [30]:
from hw4.data_preparer import shrink_repeated_samples

data = test_shuffled_data(from_file=True)
n_test_samples = 1000
queries, docs, labels = zip(*data[:n_test_samples])

queries, docs, labels = shrink_repeated_samples(
    queries=queries,
    docs=docs,
    labels=labels,
)
print(len(docs))

891


# Recall@k usage example

In [107]:
from torch import tensor
from torchmetrics.functional.retrieval.recall import retrieval_recall

preds = tensor([0.2, 0.3, 0.5])
target = tensor([True, False, True])
r2 = retrieval_recall(
    preds=preds,
    target=target,
    top_k=2
)
print(r2)

tensor(0.5000)


# Obtain preds and targets, Evaluate metrics

In [34]:
doc_embs = finetuned_bi_encoder.encode(
    docs,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

In [35]:
query_embs = finetuned_bi_encoder.encode(
    queries,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

In [57]:
cos_scores = util.semantic_search(query_embs, doc_embs, top_k=1e9)

In [58]:
print(len(cos_scores))

891


In [59]:
print(cos_scores[0])

[{'corpus_id': 0, 'score': 0.5318310856819153}, {'corpus_id': 582, 'score': 0.5177189111709595}, {'corpus_id': 178, 'score': 0.5000532269477844}, {'corpus_id': 266, 'score': 0.4995037019252777}, {'corpus_id': 51, 'score': 0.49128687381744385}, {'corpus_id': 324, 'score': 0.4910564124584198}, {'corpus_id': 659, 'score': 0.48840367794036865}, {'corpus_id': 340, 'score': 0.48133450746536255}, {'corpus_id': 837, 'score': 0.48057007789611816}, {'corpus_id': 842, 'score': 0.47911542654037476}, {'corpus_id': 809, 'score': 0.4779476821422577}, {'corpus_id': 657, 'score': 0.4765729010105133}, {'corpus_id': 884, 'score': 0.47410234808921814}, {'corpus_id': 271, 'score': 0.4688253104686737}, {'corpus_id': 725, 'score': 0.463988333940506}, {'corpus_id': 270, 'score': 0.46264612674713135}, {'corpus_id': 419, 'score': 0.46203696727752686}, {'corpus_id': 671, 'score': 0.4600253999233246}, {'corpus_id': 688, 'score': 0.4595084488391876}, {'corpus_id': 105, 'score': 0.45759814977645874}, {'corpus_id': 

In [62]:
top_k = 5

In [63]:
print('Question: ', queries[0])
print()
for no, ir in enumerate(cos_scores[0][:top_k]):
    corpus_id = ir["corpus_id"]
    print('corpus id: ', corpus_id)
    print(f'Document {no + 1}: Cosine Similarity is {ir["score"]:.3f}:\n\n{docs[corpus_id]}')
    print()

Question:  Какое содержание селена в морской воде?

corpus id:  0
Document 1: Cosine Similarity is 0.532:

Содержание селена в земной коре — около 500 мг/т. Основные черты геохимии селена в земной коре определяются близостью его ионного радиуса к ионному радиусу серы. Селен образует 37 минералов, среди которых в первую очередь должны быть отмечены ашавалит FeSe, клаусталит PbSe, тиманнит HgSe, гуанахуатит Bi2(Se, S)3, хастит CoSe2, платинит PbBi2(S, Se)3, ассоциирующие с различными сульфидами, а иногда также с касситеритом. Изредка встречается самородный селен. Главное промышленное значение на селен имеют сульфидные месторождения. Содержание селена в сульфидах колеблется от 7 до 110 г/т. Концентрация селена в морской воде 4·10−4 мг/л[15]. На территории Кавказских Минеральных Вод встречаются источники с содержанием Se от 50 мкг/дм3.

corpus id:  582
Document 2: Cosine Similarity is 0.518:

К хлоридно-гидрокарбонатно-натриевому типу относится закарпатская вода Драговская с минерализацией

In [64]:
print(docs[582])

К хлоридно-гидрокарбонатно-натриевому типу относится закарпатская вода Драговская с минерализацией 9,6 г/л и краснодарская Горячий ключ с общим содержанием солей в литре 4,5 г солей, но в них хлориды в виде поваренной соли (соответственно 59 и 67 %) превалируют над гидрокарбонатами, которые представлены содой (38 и 32 %). Обе воды безсульфатные. Преобладанием хлоридов над гидрокарбонатами отличается и вода того же типа Челкарская с минерализацией 2,2 г/л. Гидрокарбонаты в виде соды составляют 32, а хлоридов (поваренная соль) — 48 %. Кроме того, в Челкарской есть сульфаты в виде глауберовой соли (20 %).


In [65]:
n = len(queries)
print(n)

891


In [101]:
def build_target_mask_for_i(i, n):
    target_mask = [False] * n
    target_mask[i] = True

    return target_mask

In [102]:
def build_pred_mask_for_i(
        cos_scores,
        i,
        n,
):
    pred_mask = [0.0] * n
    qi_scores = cos_scores[i]

    for docid_score in qi_scores:
        doc_id = docid_score['corpus_id']
        score = docid_score['score']
        pred_mask[int(doc_id)] = score


    return pred_mask

In [103]:
def recall_k(
        preds: list,
        target: list,
        top_k: int,
):
    preds = torch.Tensor(preds)
    target = torch.Tensor(target)

    return retrieval_recall(
        preds=preds,
        target=target,
        top_k=top_k,
    )

In [104]:
r5 = recall_k(
    preds=build_pred_mask_for_i(cos_scores, i=0, n=n),
    target=build_target_mask_for_i(0, n),
    top_k=5,
)

In [105]:
print(r5)

tensor(1.)


In [115]:
from statistics import mean

r5s = []
for i in range(n):
    pred = build_pred_mask_for_i(cos_scores, i=i, n=n)
    target = build_target_mask_for_i(i=i, n=n)
    
    r5 = recall_k(preds=pred, target=target, top_k=5).item()
    r5s.append(r5)

print('mean recall5@891: ', mean(r5s))    

mean recall5@891:  0.3838383838383838
